In [ ]:
#- 패키지 임포트
import pandas as pd
import numpy as np
#import emoji # 이모지 제거용
import re
from soynlp.normalizer import * #반복되는 자음 제거
import time
from tqdm import tqdm #for문 진행률
from time import sleep #for문 진행 시간 
from kiwipiepy import Kiwi #문장 분리2

from IPython.display import clear_output
from IPython import display # 램효율을 늘리기 위해 아웃풋 display를 지워줄 것.
from ipywidgets import Output

out = Output()
display.display(out)


In [5]:
''' 파일 불러오기 '''
df = pd.read_csv("../dataset/2022/본문_크롤링/나무문화_본문_2022_3월.txt", sep='\t', encoding='UTF-8')
# print(df.shape, type(df)) # 파일 구조 및 컬럼명 확인용
df['org_idx']= df.index # 인덱스 column 만들기: 문장 분리할 때 사용함
# print(df.columns) # org_idx 제대로 추가됐나 단순 확인용.
post_list = list(df['본문']) #자연어 전처리를 위해 본문을 리스트 형태로

In [6]:
''' 불용어 제거 '''
# post_list 양이 많을 경우 불용어 제거 중에 오류 나면 다시 리스트로 만드는 데 오래 걸려서 카피하는 것임. 안 하고 post_list로 진행해도 됨.
post_list2 = post_list.copy()
for i in range(len(post_list2)):
    post_list2[i] = re.sub('\n', ' ', string=str(post_list2[i])) # 줄바꿈을 띄어쓰기 하나로
    post_list2[i] = re.compile('[^|ㄱ-ㅎ|ㅏ-ㅣ|A-Z|a-z|0-9|가-힣]+').sub(' ',post_list2[i])
    post_list2[i] = repeat_normalize(post_list2[i], num_repeats=1)
	# ↑ ㅋㅋㅋ, ㅎㅎㅎ 등의 의미 없는 자음 삭제. num_repeats가 1이면 반복 글자 중 1개만 남기고 2면 2개만 남김.
    post_list2[i] = re.sub(r"^\s+|\s+$", "", post_list2[i]) # 문서 앞뒤 공백 제거
    post_list2[i] = post_list2[i].split() # 문서 내 공백(1개 이상) 기준으로 자르기
    post_list2[i] = ' '.join(post_list2[i]) # 공백 기준으로 나뉜 문서를 다시 1개의 공백을 두고 붙임.
	# ↑ 문서 내 다중 공백을 지우기 위함.]

df['cleaning'] = pd.DataFrame(post_list2) #불용어 제거된 본문을 데이터 프레임으로

In [7]:
''' 문장 분리 '''
kiwi = Kiwi() #kss보다 속도 빠름 
split_list = [kiwi.split_into_sents(df['cleaning'][i]) for i in tqdm(range(len(df['cleaning'])))]
df['split_list'] = split_list
df

100%|██████████| 16128/16128 [46:35<00:00,  5.77it/s]  


,url,title,date,본문,org_idx,cleaning,split_list
0,https://blog.naver.com/eunreelove/222688424375,"4월 전시회 추천, 리움 미술관 순회전-인간, 일곱 개의 질문(ft....",2022.03.31.,"4월 전시회 추천, 리움 미술관 순회전-인간, 일곱 개의 질문(ft. 전남도립미술관...",0,4월 전시회 추천 리움 미술관 순회전 인간 일곱 개의 질문 ft 전남도립미술관 인간...,[(4월 전시회 추천 리움 미술관 순회전 인간 일곱 개의 질문 ft 전남도립미술관 ...
1,https://blog.naver.com/uju0612/222688424977,삼월이 갔다,2022.03.31.,브런치 먹으러 헤이러스틱\n빵은 별로 끌리는게 없어서 패스\n공간도 넓고 분위기도 ...,1,브런치 먹으러 헤이러스틱 빵은 별로 끌리는게 없어서 패스 공간도 넓고 분위기도 괜찮...,[(브런치 먹으러 헤이러스틱 빵은 별로 끌리는게 없어서 패스 공간도 넓고 분위기도 ...
2,https://blog.naver.com/xyqpi3/222688422800,[서평] 세계사를 바꾼 10가지 감염병,2022.03.31.,세계사를 바꾼 10가지 감염병\n저자\n조지무쇼\n출판\n사람과나무사이\n발매\n2...,2,세계사를 바꾼 10가지 감염병 저자 조지무쇼 출판 사람과나무사이 발매 2021 08...,[(세계사를 바꾼 10가지 감염병 저자 조지무쇼 출판 사람과나무사이 발매 2021 ...
3,https://blog.naver.com/twenty_2016/222688423411,"2022년 3월의 한 달 / 3월에 선거가 있었다, 그리고 나는 으른이...",2022.03.31.,길가에 벚꽃이 엄청엄청 많이 피었다.\n3월이 끝나가고 4월이 오면서 벚꽃이 만개하...,3,길가에 벚꽃이 엄청엄청 많이 피었다 3월이 끝나가고 4월이 오면서 벚꽃이 만개하는 ...,"[(길가에 벚꽃이 엄청엄청 많이 피었다, 0, 19, None, []), (3월이 ..."
4,https://blog.naver.com/hj9064/222688422627,밀린일상의 정리(feat.쿠사츠온천&스키여행),2022.03.31.,**움짤집착러의 데이터잡아먹는 블로그 주의**\n밀린일상의 정리(feat.쿠사츠온천...,4,움짤집착러의 데이터잡아먹는 블로그 주의 밀린일상의 정리 feat 쿠사츠온천 스키여행...,[(움짤집착러의 데이터잡아먹는 블로그 주의 밀린일상의 정리 feat 쿠사츠온천 스키...
...,...,...,...,...,...,...,...
16123,https://blog.naver.com/scgreen/222659707137,[광교 카페거리 스시 오마카세] 100% 예약제로만 운영되는...,2022.03.01.,"먹은 메뉴 : 런치 오마카세\n소비 비용 : 100,000원 (인 당 50,000)...",16123,먹은 메뉴 런치 오마카세 소비 비용 100 000원 인 당 50 000 광교에는 일...,[(먹은 메뉴 런치 오마카세 소비 비용 100 000원 인 당 50 000 광교에는...
16124,https://blog.naver.com/sheshow/222658228631,발해(9)-문왕 대흠무의 두 딸 : 정혜공주와 정효공주(774년...,2022.03.01.,blog\n발해 43개의 글\n발해\n목록닫기\n발해 글 목록\n글 제목\n작성일\...,16124,blog 발해 43개의 글 발해 목록닫기 발해 글 목록 글 제목 작성일 이정기 왕국...,[(blog 발해 43개의 글 발해 목록닫기 발해 글 목록 글 제목 작성일 이정기 ...
16125,https://blog.naver.com/rlagudtjq94/222660420287,이탈리아 왕국,2022.03.01.,블로그\n이탈리아 사보이아 가문 48개의 글\n이탈리아 사보이아 가문\n목록열기\n...,16125,블로그 이탈리아 사보이아 가문 48개의 글 이탈리아 사보이아 가문 목록열기 이탈리아...,[(블로그 이탈리아 사보이아 가문 48개의 글 이탈리아 사보이아 가문 목록열기 이탈...
16126,https://blog.naver.com/hyunsaeng1/222660515326,평창 영월 백덕산 등산코스 최단코스 (운교리 마을회관~정상),2022.03.01.,"평창 영월 백덕산 등산코스\n(백덕산/1,350m)\n블랙야크100대명산 67좌\n...",16126,평창 영월 백덕산 등산코스 백덕산 1 350m 블랙야크100대명산 67좌 백덕산 등...,[(평창 영월 백덕산 등산코스 백덕산 1 350m 블랙야크100대명산 67좌 백덕산...


In [9]:
# 데이터프레임을 10개 단위로 나누기
chunk_size = 50
num_chunks = len(df) // chunk_size + 1  # 나머지가 있는 경우 추가적인 청크 필요

dfs = {}
for i in range(num_chunks):
    start_idx = i * chunk_size
    end_idx = (i + 1) * chunk_size
    chunk_df = df[start_idx:end_idx]
    dfs[f"df_{i+1}"] = chunk_df

# 생성된 데이터프레임들 확인
for key, chunk_df in dfs.items():
    print(key)

df_1
df_2
df_3
df_4
df_5
df_6
df_7
df_8
df_9
df_10
df_11
df_12
df_13
df_14
df_15
df_16
df_17
df_18
df_19
df_20
df_21
df_22
df_23
df_24
df_25
df_26
df_27
df_28
df_29
df_30
df_31
df_32
df_33
df_34
df_35
df_36
df_37
df_38
df_39
df_40
df_41
df_42
df_43
df_44
df_45
df_46
df_47
df_48
df_49
df_50
df_51
df_52
df_53
df_54
df_55
df_56
df_57
df_58
df_59
df_60
df_61
df_62
df_63
df_64
df_65
df_66
df_67
df_68
df_69
df_70
df_71
df_72
df_73
df_74
df_75
df_76
df_77
df_78
df_79
df_80
df_81
df_82
df_83
df_84
df_85
df_86
df_87
df_88
df_89
df_90
df_91
df_92
df_93
df_94
df_95
df_96
df_97
df_98
df_99
df_100
df_101
df_102
df_103
df_104
df_105
df_106
df_107
df_108
df_109
df_110
df_111
df_112
df_113
df_114
df_115
df_116
df_117
df_118
df_119
df_120
df_121
df_122
df_123
df_124
df_125
df_126
df_127
df_128
df_129
df_130
df_131
df_132
df_133
df_134
df_135
df_136
df_137
df_138
df_139
df_140
df_141
df_142
df_143
df_144
df_145
df_146
df_147
df_148
df_149
df_150
df_151
df_152
df_153
df_154
df_155
df_156
df_157
df_158
df

In [11]:
# ''' 분리된 문장 한 개를 한 row로 '''
# df2 = pd.DataFrame(columns = ['split_str', 'org_idx']) #용량을 줄이기 위해 새로운 데이터프레임으로

# ''' 문장 데이터 프레임 저장 '''
# df2.to_csv("../dataset/2022/본문_전처리/나무문화_본문_3월_전처리.csv", index=False, encoding='UTF-8')

In [12]:
import pandas as pd
df2 = pd.read_csv("../dataset/2022/본문_전처리/나무문화_본문_3월_전처리.csv", encoding='UTF-8')
df2

,split_str,org_idx


In [ ]:
for a in range(1,len(dfs)+1):
    for i in tqdm(dfs['df_%s'%a]['split_list'].index):
        for j in range(len(dfs['df_%s'%a]['split_list'][i])):
            split_str= {'split_str': dfs['df_%s'%a]['split_list'][i][j][0], 'org_idx': [i]}
            print(split_str['org_idx'])
            split_df= pd.DataFrame(split_str)
            df2 = pd.concat([df2,split_df])
            df2.reset_index(drop=True, inplace=True)
            display.clear_output(wait=True) # 출력지우기
        else:
            time.sleep(1)
df2.reset_index(drop=True, inplace=True)
df2.to_csv("../dataset/2022/본문_전처리/나무문화_본문_3월_전처리.csv", index=False, encoding='UTF-8')
df2

In [7]:
df2.dropna(axis=0, inplace=True)
df2.reset_index(drop=True, inplace=True)
df2.to_csv("../dataset/2022/본문_전처리/나무문화_본문_3월_전처리.csv", index=False, encoding='UTF-8')